In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, max, count, sum
import random
import os

In [15]:
spark = SparkSession.builder.appName("CachingExample").getOrCreate()

# Surprass warnings
spark.sparkContext.setLogLevel("ERROR")

In [16]:
# Create a large DataFrame with user data
data = [(i, "Product" + str(random.randint(1,1000)), random.randint(1,10000), random.randint(1,20)) for i in range(1, 2000001)]
columns = ["transaction_id", "product", "customer_id", "quantity"]

transactions = spark.createDataFrame(data, columns)
transactions.show(5)

+--------------+----------+-----------+--------+
|transaction_id|   product|customer_id|quantity|
+--------------+----------+-----------+--------+
|             1|Product715|        338|      14|
|             2|Product529|       5393|      20|
|             3|Product732|       4990|      14|
|             4|Product605|       6916|       3|
|             5|Product671|       3880|      14|
+--------------+----------+-----------+--------+
only showing top 5 rows



In [17]:
avg_quantity = transactions.agg(avg("quantity")).collect()
max_quantity = transactions.agg(max("quantity")).collect()
total_quantity = transactions.agg(sum("quantity")).collect()
total_quantity_by_product = (
    transactions.groupBy(col("product")).agg(sum("quantity")).collect()
)
count_products_by_customer = (
    transactions.groupBy(col("customer_id")).agg(count("product")).collect()
)
total_transactions = transactions.count()
customer_count = transactions.select("customer_id").distinct().count()

In [18]:
transactions = transactions.cache()

avg_quantity = transactions.agg(avg("quantity")).collect()
max_quantity = transactions.agg(max("quantity")).collect()
total_quantity = transactions.agg(sum("quantity")).collect()
total_quantity_by_product = (
    transactions.groupBy(col("product")).agg(sum("quantity")).collect()
)
count_products_by_customer = (
    transactions.groupBy(col("customer_id")).agg(count("product")).collect()
)
total_transactions = transactions.count()
customer_count = transactions.select("customer_id").distinct().count()

transactions.unpersist()

DataFrame[transaction_id: bigint, product: string, customer_id: bigint, quantity: bigint]

In [19]:

# Stop the Spark session
spark.stop()